In [1]:
%reset -f
import numpy as np
import sys
from datetime import datetime
import os

In [2]:
output_log_file = "experiment_3_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

experiment_3_H.log
Sun Aug 17 14:18:30 2025
Defense rate = 0, Attack rate = 0
Target c(12,13,14,16) hit probability: 0.393203




In [3]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [4]:
attack_rate_list = [0]

In [ ]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """

    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        weights = []
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        min_weights = []
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        hardness.append(np.exp(-min_weight))
    
    hardness = np.array(hardness)
    
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))
    stop_probs = np.concatenate((1 - hardness, [1.0]))
    pdf = cumulative_probs * stop_probs

    if pdf.sum() < 1e-15:
        pdf = np.full_like(pdf, 1e-7)

    return pdf / pdf.sum()

In [6]:
%run heuristic_defense.ipynb

In [7]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())